In [5]:
import pandas as pd 
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import unidecode
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
import re

In [2]:
# Load data from a text file
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            label, sentence = line.strip().split('\t', 1)  # Split on the first tab character
            data.append([sentence, int(label)])
    return pd.DataFrame(data, columns=['sentence', 'label'])

# file_path
file_path = 'TRAINING_DATA.txt'

#assign the data into pandas DF
data = load_data(file_path)

data.head()


,sentence,label
0,"Cuando conocí a Janice en 2013 , una familia n...",1
1,Hwang habló en Sur de este año por Southwest M...,0
2,Usted podría pensar Katy Perry y Robert Pattin...,1
3,Cualquiera que haya volado los cielos del crea...,1
4,"Bueno , este cantante tendrá un LARGO tiempo p...",1


### Another method - Lemmatize and BoW

In [3]:
#include lemmatize and BoW
def preprocess_text_lemmatize(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove accents
    text = unidecode.unidecode(text)
    # Normalize numbers: Replace digits with a special token, e.g., "NUM"
    text = re.sub(r'\d+', 'NUM', text)
    #removing special characters 
    text = re.sub(r'[^\w\sáéíóúüñ¿¡0-9]', '', text) 
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text into words
    words = word_tokenize(text)
    # Define stop words for Spanish
    stop_words = set(stopwords.words('spanish'))
    # Remove stop words from the tokenized words
    words = [word for word in words if word not in stop_words]
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # Join the words back into a single string with spaces
    return ' '.join(words)

data['cleaned_setenced']= data['sentence'].apply(preprocess_text_lemmatize)

In [ ]:
data.head()

,sentence,label,cleaned_setenced
0,"Cuando conocí a Janice en 2013 , una familia n...",1,conoci janice 2013 familia necesitaba 600 punt...
1,Hwang habló en Sur de este año por Southwest M...,0,hwang hablo sur ano southwest music and medium...
2,Usted podría pensar Katy Perry y Robert Pattin...,1,usted podria pensar katy perry robert pattinso...
3,Cualquiera que haya volado los cielos del crea...,1,cualquiera volado cielos creador escuchado act...
4,"Bueno , este cantante tendrá un LARGO tiempo p...",1,bueno cantante tendra largo tiempo sentir aun ...


In [4]:
def count_vectorize(data, text_column):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(data[text_column])
    return X, vectorizer


# Logistic Regression Classification function
def logistic_regression_classification_2(X, y):
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the classifier
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(X_test)
    
    # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    return accuracy, report, classifier

# Main function to tie everything together
def main_tfidf_log_reg_2(data, text_column):
    # TF-IDF Vectorization
    X, vectorizer = count_vectorize(data, text_column)
    y = data['label']
    
    # Classification
    accuracy, report, classifier = logistic_regression_classification_2(X, y)
    
    # Output results
    print(f'Accuracy: {accuracy}')
    print(report)
    
    return vectorizer, classifier

vectorizer, classifier = main_tfidf_log_reg_2(data,'cleaned_setenced')

Accuracy: 0.3559843400447427
              precision    recall  f1-score   support

           0       0.35      0.35      0.35      1751
           1       0.37      0.36      0.36      1825

    accuracy                           0.36      3576
   macro avg       0.36      0.36      0.36      3576
weighted avg       0.36      0.36      0.36      3576



c:\Users\anama\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Trying gradient boost with count vectorize 

In [7]:
def gradient_boosting_classification_count(X, y):
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the classifier
    classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(X_test)
    
    # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    return accuracy, report, classifier

# Main function to tie everything together
def main_gradient_boosting_count(data, text_column):
    # Count Vectorization
    X, vectorizer = count_vectorize(data, text_column)
    y = data['label']
    
    # Classification
    accuracy, report, classifier = gradient_boosting_classification_count(X, y)
    
    # Output results
    print(f'Accuracy: {accuracy}')
    print(report)
    
    return vectorizer, classifier 

# Call the main function with gradient boosting
vectorizer, classifier = main_gradient_boosting_count(data, 'cleaned_setenced')

Accuracy: 0.5109060402684564
              precision    recall  f1-score   support

           0       0.50      0.13      0.21      1751
           1       0.51      0.88      0.65      1825

    accuracy                           0.51      3576
   macro avg       0.51      0.50      0.43      3576
weighted avg       0.51      0.51      0.43      3576



### Cross-Validation

In [11]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

#data definition
X= data['cleaned_setenced']
y= data['label']

#splitting the data 
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_setenced'], data['label'], test_size=0.2, random_state=42)

# Define parameter grids for both CountVectorizer and GradientBoostingClassifier
param_grid = {
    'count_vectorizer__max_features': [1000, 2000, 3000],  # Example parameter grid for CountVectorizer
    'classifier__n_estimators': [50, 100, 200],            # Example parameter grid for GradientBoostingClassifier
    'classifier__learning_rate': [0.1, 0.01, 0.001]
}

# Create the pipeline
pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('classifier', GradientBoostingClassifier())
])

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X,y)

# Get best hyperparameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print best hyperparameters and best score
print("Best hyperparameters:", best_params)
print("Best cross-validation score:", best_score)

# Evaluate the best model on a holdout test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test score of the best model:", test_score)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best hyperparameters: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 200, 'count_vectorizer__max_features': 3000}
Best cross-validation score: 0.5152430343705511
Test score of the best model: 0.5262863534675615
